In [1]:
import pandas as pd
import plotly.express as px
import warnings
import pickle
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('City_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

city_coordinates_dict=pickle.load(open('city_coordinates_dict.pickle','rb'))

In [3]:
#leave only big cities with all entries
df=df.dropna(how='any').reset_index(drop=True)
df=df.sort_values(by='SizeRank').head(500)

In [4]:
df['City_State']=df.apply(lambda x:f"{x['RegionName']} ({x['State']})", axis=1)

In [5]:
df=df.melt(id_vars=['City_State'], var_name='Date', value_vars=df.columns[df.columns.str.contains('-')], value_name='Price').sort_values(by=['City_State','Date'])

In [6]:
#create group_city df
df['Date']=pd.to_datetime(df['Date'])
grouped_city=df.groupby([df['Date'].dt.to_period('M'),'City_State'])['Price'].mean().reset_index()

In [7]:
grouped_city['Lat']=grouped_city['City_State'].apply(lambda x:city_coordinates_dict[x]['Lat'] if x in city_coordinates_dict else None)
grouped_city['Lon']=grouped_city['City_State'].apply(lambda x:city_coordinates_dict[x]['Lon'] if x in city_coordinates_dict else None)

In [8]:
grouped_city.to_csv('grouped_city.csv',index=False)

get city coordinates

from geopy.geocoders import Photon


def get_coordinates(city, state):
    geolocator =  Photon(user_agent="measurements")
    location = geolocator.geocode(f"{city}, {state}")
    if location:
        return location.latitude, location.longitude
    else:
        return None


city_coordinates_dict={}
for i, city_state in enumerate(grouped_city['City_State'].unique()):
    city=city_state.split('(')[0].strip()
    state=city_state.split('(')[-1].replace(')','').strip()
    
    lat,lon=get_coordinates(city,state)
    city_coordinates_dict[city_state]={'Lat':lat,'Lon':lon}
    
    print(i,end='\r')

pickle.dump(city_coordinates_dict,open('city_coordinates_dict.pickle','wb'))